# Aim

Get good and fast PSF fitting working, with the current models, in Python 3.8+. This could be using Theano, tf2+, or pytorch, and probably won't be using scipy.

1. (in the `oldeleanor` env) run code from master to show the baseline of fitting works
2. (in the `eleanorupgrade` env) set up frame-by-frame fitting in a new language, for sim data
3. the same as above for real data
4. set up all-at-once fitting using different weights for fluxes, and deviation-from-aperture regularization for coherence
5. the same as above for real data

In [1]:
import eleanor
import numpy as np
import lightkurve as lk
from matplotlib import pyplot as plt
import tqdm
import scipy.optimize as sopt
%load_ext autoreload
%autoreload 2

In [2]:
# first, step 1: surgically extract code from afeinstein20/eleanor/master that still operates with my new API
star = eleanor.Source(tic=120362128, sector=14, tc=True)
star.premade = False
sc = lk.search_targetpixelfile(target='tic120362128', sector=14).download()
sq = sc.quality < 5000
start = 2500
end = 3400
time = sc.time[sq][start:end].value
tpfs = sc.flux[sq][start:end].value
errs = sc.flux_err[sq][start:end].value
bkgs = sc.flux_bkg[sq,0,0][start:end].value
bkg0 = np.mean(bkgs)
data = eleanor.TargetData(
    star, 
    height=11, 
    width=11, 
    do_pca=True, 
    do_psf=False,
)

In [3]:
data.psf_lightcurve(data_arr=tpfs, err_arr=errs, bkg_arr=bkgs, verbose=True, nstars=3, xc=[4.9, 4.5, 4.7], yc=[3.0, 4.4, 7.0], ignore_pixels=1)

AttributeError: module 'tensorflow' has no attribute 'logging'

In [4]:
plt.plot(data.psf_flux[:,0])

TypeError: 'NoneType' object is not subscriptable

In [5]:
data.psf_params

AttributeError: 'TargetData' object has no attribute 'psf_params'

In [6]:
xc = [4.9, 4.5, 4.7] 
yc = [3.0, 4.4, 7.0]
model = eleanor.models.Gaussian(
    shape=tpfs.shape[1:],
    col_ref = 0,
    row_ref = 0,
    xc = xc,
    yc = yc,
    fit_idx = 0,
    bkg0 = np.max(bkgs[0])
)
nstars = len(xc)
fout = np.zeros((len(tpfs), 3))
lout = np.zeros(len(tpfs))
pars0 = model.get_default_par(tpfs[0])

tpfs = np.array(tpfs, dtype=np.float32)
errs = np.array(errs, dtype=np.float32)

import jax.numpy as jnp
from jax import grad # remember to install jaxlib

loss = lambda mean_val, i: jnp.sum((mean_val - tpfs[i]) ** 2 / errs[i])

def loss_fn(params, i):
    fluxes = params[:nstars]
    xshift, yshift, bkg = params[nstars:nstars+3]
    optpars = params[nstars+3:]
    mean_val = model.mean(fluxes, xshift, yshift, bkg, optpars)
    return loss(mean_val, i)

loss_fn(pars0, 0)

DeviceArray(40513670., dtype=float32)

In [7]:
grad(loss_fn)(pars0, 3)

DeviceArray([-4.8649893e+02, -9.1114246e+02, -5.0716370e+02,
              2.2673736e+07,  7.3639720e+06, -9.1125898e+03,
             -2.7456202e+06,  1.8855470e+06,  1.3953994e+05],            dtype=float32)

In [10]:
grad_fn = grad(loss_fn)

In [11]:
grad_fn(pars0, 3)

DeviceArray([-4.8649893e+02, -9.1114246e+02, -5.0716370e+02,
              2.2673736e+07,  7.3639720e+06, -9.1125898e+03,
             -2.7456202e+06,  1.8855470e+06,  1.3953994e+05],            dtype=float32)

In [12]:
from torch.autograd import grad as tgrad

In [13]:
help(tgrad)

Help on function grad in module torch.autograd:

grad(outputs: Union[torch.Tensor, Sequence[torch.Tensor]], inputs: Union[torch.Tensor, Sequence[torch.Tensor]], grad_outputs: Union[torch.Tensor, Sequence[torch.Tensor], NoneType] = None, retain_graph: Union[bool, NoneType] = None, create_graph: bool = False, only_inputs: bool = True, allow_unused: bool = False) -> Tuple[torch.Tensor, ...]
    Computes and returns the sum of gradients of outputs w.r.t. the inputs.
    
    ``grad_outputs`` should be a sequence of length matching ``output``
    containing the "vector" in Jacobian-vector product, usually the pre-computed
    gradients w.r.t. each of the outputs. If an output doesn't require_grad,
    then the gradient can be ``None``).
    
    If ``only_inputs`` is ``True``, the function will only return a list of gradients
    w.r.t the specified inputs. If it's ``False``, then gradient w.r.t. all remaining
    leaves will still be computed, and will be accumulated into their ``.grad``
 

In [14]:
def loss_and_grad(pars, i):
    return loss_fn(pars, i), grad_fn(pars, i)

In [15]:
res = sopt.minimize(loss_and_grad, pars0, 0, jac=True, method='TNC', tol=1e-2)

In [21]:
sopt.minimize(loss_and_grad, res.x, 1, jac=True, method='TNC', tol=1e-2)

     fun: DeviceArray(2585573.5, dtype=float32)
     jac: array([-2.31948423e+00, -1.05771911e+00, -4.02379704e+00, -1.22112062e+05,
        1.38084188e+05, -1.97105420e+03,  1.20065875e+05, -7.72302500e+04,
        3.45133125e+04], dtype=float32)
 message: 'Converged (|x_n-x_(n-1)| ~= 0)'
    nfev: 33
     nit: 6
  status: 2
 success: True
       x: array([ 1.44533330e+04,  6.50601928e+04,  4.09625208e+04, -3.85942024e-01,
       -5.74232261e-01,  3.87870895e+01,  7.05756095e-01, -1.09814346e-01,
        7.98810856e-01])

In [19]:
%timeit grad_fn(pars0, 1)

178 ms ± 66.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit loss_fn(pars0, 1)

6.92 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
sopt.minimize(loss_fn, pars0, 0, method='TNC', tol=1e-4)

     fun: DeviceArray(40513670., dtype=float32)
     jac: array([0.e+00, 0.e+00, 0.e+00, 4.e+08, 0.e+00, 0.e+00, 4.e+08, 0.e+00,
       0.e+00])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 210
     nit: 1
  status: 1
 success: True
       x: array([ 1.58314727e+04,  1.58314727e+04,  1.58314727e+04, -1.27105749e-14,
        0.00000000e+00,  1.57312119e+02,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00])

In [25]:
tgrad

<function torch.autograd.grad(outputs: Union[torch.Tensor, Sequence[torch.Tensor]], inputs: Union[torch.Tensor, Sequence[torch.Tensor]], grad_outputs: Union[torch.Tensor, Sequence[torch.Tensor], NoneType] = None, retain_graph: Union[bool, NoneType] = None, create_graph: bool = False, only_inputs: bool = True, allow_unused: bool = False) -> Tuple[torch.Tensor, ...]>

In [22]:
pars = pars0
for i in tqdm.trange(900):
    res = sopt.minimize(loss_and_grad, pars, i, jac=True, method='TNC', tol=1e-4)
    pars = res.x
    if i % 100 == 0:
        print(res.x)

  0%|          | 1/900 [00:12<3:03:15, 12.23s/it][ 1.49233422e+04  6.44869032e+04  4.04104893e+04 -3.71329166e-01
 -5.77347610e-01  5.46118120e+01  7.15253486e-01 -1.19242512e-01
  8.12071524e-01]
  2%|▏         | 20/900 [05:47<4:14:34, 17.36s/it]


KeyboardInterrupt: 

In [87]:
pars0

array([1.58314727e+04, 1.58314727e+04, 1.58314727e+04, 0.00000000e+00,
       0.00000000e+00, 1.57312119e+02, 1.00000000e+00, 0.00000000e+00,
       1.00000000e+00])

In [79]:
res = sopt.minimize(fun=loss_fn, x0=pars.detach().numpy(), jac=True, method='L-BFGS-B')

AttributeError: 'numpy.ndarray' object has no attribute 'split'

In [ ]:

for (tpf, err) in zip(tpfs_t, errs_t):
    loss_fn = get_loss_fn(tpf, err)

    @tf.function
    def val_and_grad(x):
        with tf.GradientTape() as tape:
            tape.watch(x)
            loss = loss_fn(x)
        grad = tape.gradient(loss, x)
        return loss, grad

    @tf.function
    def func(x):
        return [vv.numpy().astype(np.float64) for vv in val_and_grad(tf.constant(x, dtype=tf.float32))]

    resdd = sopt.minimize(fun=func, x0=pars, jac=True, method='L-BFGS-B')